## Import model

In [253]:
## import libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets

from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.tensorboard import SummaryWriter

CUDA = torch.cuda.is_available()
# device = "cpu"
device = torch.device("cuda" if CUDA else "cpu")

## Prep dataset

In [254]:
import sys
# run the below line once only
if "..\\chexnet" not in sys.path:
    sys.path.insert(0,r'..\chexnet')
print(sys.path)

['..\\chexnet', 'c:\\Users\\siyang\\Documents\\GitHub\\DeepLearningProject\\notebooks', 'C:\\Python312\\python312.zip', 'C:\\Python312\\DLLs', 'C:\\Python312\\Lib', 'C:\\Python312', 'c:\\Users\\siyang\\Documents\\GitHub\\DeepLearningProject\\.venv', '', 'c:\\Users\\siyang\\Documents\\GitHub\\DeepLearningProject\\.venv\\Lib\\site-packages', 'C:\\Users\\siyang\\Documents\\GitHub\\DeepLearningProject', 'c:\\Users\\siyang\\Documents\\GitHub\\DeepLearningProject\\.venv\\Lib\\site-packages\\win32', 'c:\\Users\\siyang\\Documents\\GitHub\\DeepLearningProject\\.venv\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\siyang\\Documents\\GitHub\\DeepLearningProject\\.venv\\Lib\\site-packages\\Pythonwin']


In [255]:
from DatasetGenerator import DatasetGenerator

In [256]:
pathDirData = r'C:\Users\siyang\Documents\GitHub\DeepLearningProject\raw_data\archive'
pathFileTrain = r'C:\Users\siyang\Documents\GitHub\DeepLearningProject\chexnet\dataset\train_1.txt'
pathFileVal = r'C:\Users\siyang\Documents\GitHub\DeepLearningProject\chexnet\dataset\val_1.txt'

transResize = 256
transCrop = 224
trBatchSize = 32
num_class = 14

normalize = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

torch.manual_seed(42)

transformList = []
transformList.append(transforms.Resize(transResize))
transformList.append(transforms.RandomResizedCrop(transCrop))
transformList.append(transforms.RandomHorizontalFlip())
transformList.append(transforms.ToTensor())
transformList.append(normalize)      
transformSequence=transforms.Compose(transformList)

datasetTrain = DatasetGenerator(pathImageDirectory=pathDirData, pathDatasetFile=pathFileTrain, transform=transformSequence)
datasetVal =   DatasetGenerator(pathImageDirectory=pathDirData, pathDatasetFile=pathFileVal, transform=transformSequence)
train_loader = DataLoader(dataset=datasetTrain, batch_size=trBatchSize, shuffle=True,  num_workers=12, pin_memory=True)
val_loader = DataLoader(dataset=datasetVal, batch_size=trBatchSize, shuffle=False, num_workers=12, pin_memory=True)

Collected 2048 images from C:\Users\siyang\Documents\GitHub\DeepLearningProject\chexnet\dataset\train_1.txt
Collected 2048 images from C:\Users\siyang\Documents\GitHub\DeepLearningProject\chexnet\dataset\val_1.txt


## Create your own DenseNet

In [257]:
class dense_layer(nn.Module):
    def __init__(self, dim):
        super(dense_layer, self).__init__()
        eps = 1e-5
        momentum = 0.1
        hidden_dim = 128
        output_dim = 32
        self.net = nn.Sequential(
            nn.BatchNorm2d(num_features=dim, eps=eps, momentum=momentum, affine=True, track_running_stats=True),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=dim, out_channels=hidden_dim, kernel_size=(1,1), stride=(1,1),bias=False),
            nn.BatchNorm2d(num_features=hidden_dim, eps=eps, momentum=momentum, affine=True, track_running_stats=True),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=hidden_dim, out_channels=output_dim, kernel_size=(1,1), stride=(1,1),bias=False)
        )
        
    def forward(self, x):
        return self.net(x)     

In [258]:
class dense_block(nn.Module):
    def __init__(self, layer_count, dim):
        super(dense_block, self).__init__()
        hidden_dim = dim
        for layer_index in range(layer_count):
            setattr(self, f'layer_{layer_index}',dense_layer(hidden_dim))
            ## update the hidden_dim so that the input size of the next layer has space
            ## for all the outputs that precede it
            hidden_dim += getattr(self,f'layer_{layer_index}').net[5].out_channels

    def forward(self, input):
        output = input
        for layer in self.children():
            new_output = layer(output)
            ## add the new_output to the previous output
            ## for the next layer in the block
            output = torch.concat((output,new_output),1) 
        return output

In [259]:
class transition_block(nn.Module):
    def __init__(self, channels):
        super(transition_block, self).__init__()
        eps = 1e-5
        momentum = 0.1
        self.norm = nn.BatchNorm2d(channels, eps=eps, momentum=momentum, affine=True, track_running_stats=True)
        self.relu = nn.ReLU()
        self.conv = nn.Conv2d(channels, channels//2, kernel_size=(1,1), stride=(1,1), bias=False)
        self.pool = nn.AvgPool2d(kernel_size=2,stride=2,padding=0)

    def forward(self, input):
        x = input
        for layer in self.children():
            x = layer(x)
        return x


In [260]:
class dense_net(nn.Module):
    def __init__(self, num_class):
        super(dense_net, self).__init__()
        hidden_dim = 64

        self.initial_setup = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=hidden_dim, kernel_size=(7,7), stride=(2,2), padding=(3,3), bias=False),
            nn.BatchNorm2d(num_features=hidden_dim, eps=1e-5, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3,stride = 2,padding=1, dilation=1, ceil_mode=False)
        )

        ## just calculate the number of channels in trans blocks manually
        ## since we know initial channel number and how many layers are in each block
        self.denseblock1 = dense_block(layer_count=6, dim=hidden_dim)
        self.trans1 = transition_block(channels = hidden_dim + 6 * 32) # 256 
        self.denseblock2 = dense_block(layer_count=12, dim=hidden_dim * 2)
        self.trans2 = transition_block(channels = hidden_dim * 2 + 12 * 32) #512
        self.denseblock3 = dense_block(layer_count=24, dim=hidden_dim * 4)
        self.trans3 = transition_block(channels = hidden_dim * 4 + 24 * 32) # 1024
        self.denseblock4 = dense_block(layer_count=16, dim=hidden_dim * 8)
        self.norm5 = nn.BatchNorm2d(num_features=hidden_dim * 8 + 16 * 32)

        ## preparing input into the classifier
        self.adaptive_avg_pool = nn.AdaptiveAvgPool2d((1,1)) ## global average pooling
        self.flatten = nn.Flatten()

        ## classifier component:
        self.classifier = nn.Sequential(
            nn.Linear(in_features=self.norm5.num_features, out_features=num_class, bias=True),
            nn.Sigmoid()
        )

    def forward(self, input):
        x = input
        for block in self.children():
            # print("working in " + str(block._get_name()) + " ...")
            x = block(x)
            # print("output of this block is " + str(x.shape))
        return x

In [261]:
model = dense_net(14)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
model.to(device)

dense_net(
  (initial_setup): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (denseblock1): dense_block(
    (layer_0): dense_layer(
      (net): Sequential(
        (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): ReLU(inplace=True)
        (2): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (4): ReLU(inplace=True)
        (5): Conv2d(128, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
    )
    (layer_1): dense_layer(
      (net): Sequential(
        (0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): ReLU(inplace=True)
 

## Train Model

In [262]:
## function to calculate the F1 score
def f1_score(tp, fp, fn):
    return 2 * (tp) / (2 * tp + fp + fn)

In [263]:
# Define the loss function and optimizer
criterion = nn.BCELoss(reduction='mean')
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
scheduler = ReduceLROnPlateau(optimizer, factor = 0.1, patience = 5, mode = 'min')

# Create a TensorBoard writer
writer = SummaryWriter(log_dir=r"./dnet_runs")

# Train the model
n_epochs = 5
model.train()
for epoch in range(n_epochs):

    for i, (images, labels, _) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        print(images.shape)
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # calculate statistics
        # pred_labels = (nn.Softmax(dim=1)(outputs) > 1/14).long()
        
        tp_array = [0 for x in range(num_class)]
        fp_array = [0 for x in range(num_class)]
        fn_array = [0 for x in range(num_class)]
        pred_labels = (outputs > 1/2).long()
        tp_array += sum(torch.logical_and(pred_labels, labels))
        fp_array += sum(torch.logical_and(torch.logical_xor(pred_labels, labels).long(), pred_labels))
        fn_array += sum(torch.logical_and(torch.logical_xor(pred_labels, labels).long(), labels))
        
        writer.add_scalar('Loss/train', loss, epoch * len(train_loader) + i)
        writer.add_scalar('TP_Sum/train', sum(tp_array), epoch * len(train_loader) + i)
        writer.add_scalar('FP_Sum/train', sum(fp_array), epoch * len(train_loader) + i)
        writer.add_scalar('FN_Sum/train', sum(fn_array), epoch * len(train_loader) + i)
        writer.add_scalar('F1_Score/train', f1_score(sum(tp_array), sum(fp_array), sum(fn_array)), epoch * len(train_loader) + i)

        # Backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        scheduler.step(loss)

        # Display
        # if (i + 1) % 100 == 0:
        print("Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, tp_sum: {:.4f}, fp_sum: {:.4f}, fn_sum: {:.4f}, cumulative_f1_score: {:.4f}".format(epoch + 1, \
                                                                     n_epochs, \
                                                                     i + 1, \
                                                                     len(train_loader), \
                                                                     loss,\
                                                                     sum(tp_array), \
                                                                     sum(fp_array),\
                                                                     sum(fn_array),\
                                                                     f1_score(sum(tp_array), sum(fp_array), sum(fn_array))))
        print("outputs\n", outputs)
        print("pred_labels\n", pred_labels)
        print("actual labels\n", labels)
    

torch.Size([32, 3, 224, 224])
Epoch [1/5], Step [1/64], Loss: 0.7015, tp_sum: 13.0000, fp_sum: 217.0000, fn_sum: 9.0000, cumulative_f1_score: 0.1032
outputs
 tensor([[0.5151, 0.5114, 0.5058, 0.5256, 0.4591, 0.5447, 0.4955, 0.5470, 0.5270,
         0.4651, 0.5337, 0.5200, 0.4832, 0.5156],
        [0.5115, 0.4890, 0.5076, 0.4933, 0.5115, 0.4968, 0.4741, 0.4592, 0.4827,
         0.4867, 0.5097, 0.5177, 0.5120, 0.4896],
        [0.5000, 0.5765, 0.5757, 0.5279, 0.4736, 0.5083, 0.5532, 0.4952, 0.4562,
         0.5085, 0.5461, 0.5501, 0.4908, 0.5126],
        [0.4204, 0.5451, 0.3983, 0.4249, 0.4623, 0.5009, 0.4151, 0.3732, 0.4241,
         0.5279, 0.4214, 0.5772, 0.4875, 0.5737],
        [0.4812, 0.4800, 0.5315, 0.4846, 0.5411, 0.4744, 0.5061, 0.5341, 0.5180,
         0.4727, 0.5059, 0.4019, 0.5453, 0.4501],
        [0.4978, 0.4583, 0.5189, 0.4741, 0.5257, 0.4678, 0.4890, 0.4389, 0.5609,
         0.4978, 0.4941, 0.5096, 0.5368, 0.4914],
        [0.5656, 0.4833, 0.4281, 0.4304, 0.4832, 0.4664,